In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import plotly.express as px
import duckdb
import os
from pathlib import Path
from typing import List
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [13]:
# Afficher le répertoire de travail actuel
import os
print("Répertoire de travail actuel:", os.getcwd())

# Lister les fichiers et dossiers du répertoire parent
print("\nContenu du répertoire parent (..):")
print(os.listdir(".."))

# Lister les fichiers et dossiers deux niveaux au-dessus
print("\nContenu deux niveaux au-dessus (../..):")
try:
    print(os.listdir("../.."))
except FileNotFoundError:
    print("Impossible d'accéder à ce répertoire")

Répertoire de travail actuel: /home/c-enjalbert/Documents/EPSI/MSPR/bloc_2/amazing/ml_ia/item-based-clustering

Contenu du répertoire parent (..):
['item-based-clustering', 'analyse_exploratoire.ipynb', 'clustering']

Contenu deux niveaux au-dessus (../..):
['amazing.duckdb', 'ml_ia', '.amazing.duckdb', 'requirements.txt', '.git', 'Access_token.txt', 'README.md', 'etl', 'data', '.gitignore']


In [14]:
# Essayez différentes approches pour localiser le dossier data
possible_paths = [
    Path("../data"),             # Un niveau au-dessus
    Path("../../data"),          # Deux niveaux au-dessus
    Path("../../../data"),       # Trois niveaux au-dessus
    Path("../../../../data"),    # Quatre niveaux au-dessus
]

for path in possible_paths:
    if path.is_dir():
        print(f"Dossier data trouvé à : {path.resolve()}")
        data_folder = path
        break
else:
    print("Impossible de trouver le dossier data automatiquement")

Dossier data trouvé à : /home/c-enjalbert/Documents/EPSI/MSPR/bloc_2/amazing/data


### Connexion à la DB / Import des Data


In [15]:
# Store database at project root
DB_NAME = Path("/home/c-enjalbert/Documents/EPSI/MSPR/bloc_2/amazing/amazing.duckdb") 
# Go up one level from current directory to get to project root
data_folder = Path("../..").resolve() / "data"
# For absolute certainty, you could use the absolute path
# data_folder = Path("/home/c-enjalbert/Documents/EPSI/MSPR/bloc_2/amazing/data")
con = duckdb.connect(str(DB_NAME))

In [16]:
# 2. Query to list all tables in the database
# DuckDB specific way to list tables
tables_info = con.sql("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
    ORDER BY table_name
""").df()

print(f"Found {len(tables_info)} tables in the database:\n")

if len(tables_info) > 0:
    for i, table_name in enumerate(tables_info['table_name']):
        print(f"{i+1}. {table_name}")
else:
    print("No tables found in the database.")

Found 2 tables in the database:

1. all_events
2. loaded_files


In [17]:
# 5. Alternative way to show all tables
print("List of all tables using DuckDB's connections.tables():")
con.sql("SHOW TABLES").show()

List of all tables using DuckDB's connections.tables():
┌──────────────┐
│     name     │
│   varchar    │
├──────────────┤
│ all_events   │
│ loaded_files │
└──────────────┘



### Save Tables as DataFrame

In [18]:
# Get event counts by event_type
print("Event counts by type:")
all_events = con.sql("""
    SELECT 
    *
    FROM all_events
    LIMIT 20000
""")
all_events.show()



Event counts by type:
┌─────────────────────┬────────────┬────────────┬─────────────────────┬─────────────────────────────────────┬──────────┬─────────┬───────────┬──────────────────────────────────────┐
│     event_time      │ event_type │ product_id │     category_id     │            category_code            │  brand   │  price  │  user_id  │             user_session             │
│      timestamp      │  varchar   │  varchar   │       varchar       │               varchar               │ varchar  │ double  │  varchar  │               varchar                │
├─────────────────────┼────────────┼────────────┼─────────────────────┼─────────────────────────────────────┼──────────┼─────────┼───────────┼──────────────────────────────────────┤
│ 2019-10-01 00:00:00 │ view       │ 44600062   │ 2103807459595387724 │ NULL                                │ shiseido │   35.79 │ 541312140 │ 72d76fde-8bb3-4e00-8c23-a032dfed738c │
│ 2019-10-01 00:00:00 │ view       │ 3900821    │ 205301355232677090

In [19]:
all_events_df = pd.DataFrame(all_events.df())
all_events_df.describe(include="all").T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
event_time,20000,NaN,NaN,NaN,2019-10-01 02:29:27.043549,2019-10-01 00:00:00,2019-10-01 02:28:14.750000,2019-10-01 02:37:18,2019-10-01 02:46:00,2019-10-01 02:54:01,NaN
event_type,20000,3,view,19548,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_id,20000,7178,1004856,234,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category_id,20000,443,2053013555631882655,5387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category_code,13412,115,electronics.smartphone,5387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
brand,17185,922,samsung,2330,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,20000.0,NaN,NaN,NaN,289.381401,0.0,60.23,154.42,359.08,2574.07,368.18694
user_id,20000,5012,531063605,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_session,20000,5669,f85114d0-6167-4c8b-88f8-0cd1cfc2c10c,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Normalisation et Préparation de la Data 

In [20]:
all_events_df.columns

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [22]:
allevents_df_CF = all_events_df[["user_id", "product_id", "category_code", "category_id"]]

In [ ]:
con.close()